# A simple test visualizing a single OMI SO2 file for a specific sub-area

## Preparation for something more useful

Based largely on https://arset.gsfc.nasa.gov/sites/default/files/airquality/webinars/18-hires/s3_omi_data_download-final.pdf and https://arset.gsfc.nasa.gov/airquality/python-scripts-aerosol-data-sets-merra-modis-and-omi

In [ ]:
import h5py
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
import sys
import os
import glob
import cartopy.crs as ccrs
from PIL import Image

In [ ]:
#vanuatu
# plot_extent = [161, 179, -22, -8]
# title_x = 161.5
# title_y = -9
# title_text = 'SO2 (3 km plume centre) - Vanuatu - date'
# imlat = -20.75
# imlon = 178.25
# imlon -= 180
# xlocs=[158,162,166,170,174,178,182]
# ylocs=[-6,-10,-14,-18,-22]
# f = '2020-03-10/OMI-Aura_L2-OMSO2_2020m0310t0142-o83251_v003-2020m0310t144039.he5'

#new zealand
# plot_extent = [170, 180, -42, -33]
# title_x = 170.25
# title_y = -33.5
# title_text = 'SO2 (3 km plume centre) - New Zealand - date'
# imlat = -41.25
# imlon = 179.5
# imlon -= 180
# xlocs=[168,172,176,180]
# ylocs=[-42,-40,-36,-32]
# f = '2019-12-09/OMI-Aura_L2-OMSO2_2019m1209t0111-o81911_v003-2020m0223t130953.he5'

#samoa-fiji-tonga
# plot_extent = [172, 192, -24, -10]
# title_x = 172.5
# title_y = -11
# title_text = 'SO2 (3 km plume centre) - Samoa-Fiji-Tonga - date'
# imlat = -22
# imlon = 190
# imlon -= 180
# xlocs=[172,176,180,-180,-176,-172,-168]
# ylocs=[-24,-20,-16,-12,-10]
# f = '2014-12-24/OMI-Aura_L2-OMSO2_2014m1224t0040-o55538_v003-2020m0221t181616.he5'

#kermadecs
# plot_extent = [172, 188, -33, -21]
# title_x = 172.5
# title_y = -22
# title_text = 'SO2 (3 km plume centre) - Kermadec Islands - date'
# imlat = -30
# imlon = 187
# imlon -= 180
# xlocs=[172,176,180,-180,-176,-172]
# ylocs=[-33,-32,-26,-22,-21]
# f = '2019-12-09/OMI-Aura_L2-OMSO2_2019m1209t0111-o81911_v003-2020m0223t130953.he5'

#png-solomons
plot_extent = [142, 164, -14, 0]
title_x = 142.5
title_y = -1
title_text = 'SO2 (3 km plume centre) - PNG-Solomons - date'
imlat = -12.5
imlon = 163
imlon -= 180
xlocs=[142,144,148,152,156,160,164]
ylocs=[-14,-10,-6,-2,0]
f = '2019-10-06/OMI-Aura_L2-OMSO2_2019m1006t0255-o80980_v003-2020m0223t103101.he5'

In [ ]:
file = h5py.File(f, 'r')

dataFields=file['HDFEOS']['SWATHS']['OMI Total Column Amount SO2']['Data Fields']
geolocation=file['HDFEOS']['SWATHS']['OMI Total Column Amount SO2']['Geolocation Fields']
SDS_NAME='ColumnAmountSO2_TRL'
data=dataFields[SDS_NAME]

valid_min=data.attrs['ValidRange'][0]
valid_max=data.attrs['ValidRange'][1]
map_label=data.attrs['Units'].decode()

#get necessary attributes 
fv=data.attrs['_FillValue']
mv=data.attrs['MissingValue']
try:
    offset=data.attrs['Offset']
except:
    offset=np.array([0],dtype='float')
try:
    scale=data.attrs['ScaleFactor']
except:
    scale=np.array([1],dtype='float')

#get lat and lon information 
lat=geolocation['Latitude'][:]
limit = -90
b = np.ma.MaskedArray(lat, lat<limit)
min_lat=np.min(b)
max_lat=np.max(b)
lon=geolocation['Longitude'][:]
limit = -180
b = np.ma.MaskedArray(lon, lon<limit)
min_lon=np.min(b)
max_lon=np.max(b)

#get the data as an array and mask fill/missing values
dataArray=data[:]
dataArray[dataArray==fv]=np.nan
dataArray[dataArray==mv]=np.nan
dataArray = scale * (dataArray - offset)

#get statistics about data
average=np.nanmean(dataArray)
stdev=np.nanstd(dataArray)
median=np.nanmedian(dataArray)

#print statistics 
# print('The average of this data is: ',round(average,3),'\nThe standard deviation is: ',round(stdev,3),'\nThe median is: ',round(median,3))
# print('The range of latitude in this file is: ',min_lat,' to ',max_lat, 'degrees \nThe range of longitude in this file is: ',min_lon, ' to ',max_lon,' degrees')
# print()

dataArray = np.ma.masked_array(dataArray, np.isnan(dataArray))

#plot
fig = plt.figure(figsize=(15,15))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
ax.set_extent(plot_extent)
ax.coastlines(resolution='10m')
ax.gridlines(draw_labels=True, xlocs=xlocs, ylocs=ylocs)

ax.text(title_x, title_y, title_text, transform=ccrs.PlateCarree(), horizontalalignment='left', fontsize=14)

#GNS logo
img = Image.open('GNS_logo.png')
imagebox = OffsetImage(img, zoom=0.075)
imagebox.image.axes = ax
ab = AnnotationBbox(imagebox, [imlon, imlat], pad=0, frameon=False)
ax.add_artist(ab)

#missing data, when data array contains the missing or fill value (the same value)
missing = data[data[:]==mv]
lat_missing = lat[data[:]==mv]
lon_missing = lon[data[:]==mv]
ax.plot(lon_missing, lat_missing, marker='.', color='red', markersize=8, alpha=0.05, transform=ccrs.PlateCarree())

#contour data
cs = plt.contourf(lon, lat, dataArray, levels=[0, 0.8, 1.5, 2.2, 3.0], colors=['white', 'purple', 'green', 'yellow', 'red'],
                 extend='max', alpha=0.5, transform=ccrs.PlateCarree())
fig.colorbar(cs, ax=ax, shrink=0.5, label='Total column SO2 (DU)')

#save plot
plt.gcf()
plt.savefig('test.png')

file.close()